In [3]:
import getpass
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", convert_system_message_to_human=True)
input_msg = input("請輸入問題")
result = model.invoke(
    [
        SystemMessage(content="請使用繁體中文回答。"),
        HumanMessage(content=input_msg),
    ]
)
display(Markdown(result.content))



c:\Users\26855\anaconda3\envs\AI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\26855\Desktop\Github\AI\langchain\langchain.py:49: SyntaxWarning: invalid escape sequence '\]'
  .replace("\]", "]")
c:\Users\26855\Desktop\Github\AI\langchain\langchain.py:50: SyntaxWarning: invalid escape sequence '\['
  .replace("\[", "[")
c:\Users\26855\anaconda3\envs\AI\Lib\site-packages\langchain_google_genai\chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**人工智慧 (AI)** 是一種模擬人類智慧的電腦科學領域。它涉及開發能夠執行通常需要人類智慧才能完成的任務的系統。

**AI 的主要特徵包括：**

* **學習：** AI 系統可以從數據中學習，並隨著時間推移改善其性能。
* **推理：** AI 系統可以進行邏輯推理，得出結論並解決問題。
* **決策：** AI 系統可以根據數據和推理做出決策。
* **自然語言處理：** AI 系統可以理解和生成人類語言。
* **電腦視覺：** AI 系統可以「看到」和理解圖像。

**AI 的應用領域十分廣泛，包括：**

* **自動駕駛汽車**
* **虛擬助理**
* **醫療診斷**
* **金融交易**
* **客戶服務**
* **遊戲**

**AI 的類型：**

* **狹義人工智慧 (Narrow AI)：** 針對特定任務設計的 AI 系統，例如面部識別或語音轉文字。
* **通用人工智慧 (General AI)：** 具有與人類相同的認知能力，能夠執行任何智力任務的 AI 系統。
* **超級人工智慧 (Super AI)：** 超越人類智慧的 AI 系統，能夠解決人類無法解決的問題。

**AI 的未來：**

AI 的發展正在迅速進行，預計將在未來繼續對社會產生深遠的影響。AI 將繼續自動化任務、改進決策過程，並為人類創造新的機會。


In [6]:
from langchain_core.runnables import RunnableLambda
from functools import partial
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Union
from operator import itemgetter

from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

# Useful Tools
def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))


In [4]:
## Zero-shot classification prompt and chain w/ explicit few-shot prompting
sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

## Define your simple instruct_model
instruct_chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash", convert_system_message_to_human=True)
instruct_llm = instruct_chat | StrOutputParser()
one_word_llm = instruct_chat.bind(stop=[" ", "\n"]) | StrOutputParser() # stop the output

zsc_chain = zsc_prompt | one_word_llm

## Function that just prints out the first word of the output. With early stopping bind
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print(zsc_call("Should I take the next exit, or keep going to the next one?"))

car


In [5]:
gen_prompt = ChatPromptTemplate.from_template(
    "Make a new sentence about the the following topic: {topic}. Be creative!"
)

gen_chain = gen_prompt | instruct_llm

input_msg = "I get seasick, so I think I'll pass on the trip"
options = ["car", "boat", "airplane", "bike"]

chain = (
    ## -> {"input", "options"}
    {'topic' : zsc_chain}
    | PPrint()
    ## -> {**, "topic"}
    | gen_chain
    ## -> string
)

chain.invoke({"input" : input_msg, "options" : options})

State:  {'topic': 'boat'}


'The old sailboat, its weathered hull whispering tales of distant shores, dreamt of escaping its moorings and dancing with the wind once more. \n'

In [1]:
from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial

big_chain = (
    PPrint()
    ## Manual mapping. Can be useful sometimes and inside branch chains
    | {'input' : lambda d: d.get('input'), 'topic' : zsc_chain}
    | PPrint()
    ## RunnableAssign passing. Better for running state chains by default
    | RunnableAssign({'generation' : gen_chain})
    | PPrint()
    ## Using the input and generation together
    | RunnableAssign({'combination' : (
        ChatPromptTemplate.from_template(
            "Consider the following passages:"
            "\nP1: {input}"
            "\nP2: {generation}"
            "\n\nCombine the ideas from both sentences into one simple one."
        )
        | instruct_llm
    )})
)

output = big_chain.invoke({
    "input" : "I get seasick, so I think I'll pass on the trip",
    "options" : ["car", "boat", "airplane", "bike", "unknown"]
})
pprint("Final Output: ", output)

c:\Users\26855\anaconda3\envs\AI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\26855\Desktop\Github\AI\langchain\langchain.py:49: SyntaxWarning: invalid escape sequence '\]'
  .replace("\]", "]")
c:\Users\26855\Desktop\Github\AI\langchain\langchain.py:50: SyntaxWarning: invalid escape sequence '\['
  .replace("\[", "[")


ModuleNotFoundError: No module named 'langchain.schema'; 'langchain' is not a package